In [4]:
import pandas as pd
import numpy as np
from pathlib import Path

# --- 1. CONFIGURATION ---
PROJECT_ROOT = Path("/home/onyxia/work/Gestion-portefeuille/")
INPUT_PATH = PROJECT_ROOT / "data/interim/cac40_interim_features.csv"
CLUSTERS_PATH = PROJECT_ROOT / "data/processed/clusters_mapping.csv"
OUTPUT_PATH = PROJECT_ROOT / "data/processed/cac40_final_superset.csv"

# --- 2. CHARGEMENT ---
df = pd.read_csv(INPUT_PATH, parse_dates=['Date'])
# On charge le mapping des clusters sauvegardé au Notebook 03
df_clusters = pd.read_csv(CLUSTERS_PATH, index_col='Ticker')
cluster_map = df_clusters['Cluster'].to_dict()

# --- 3. INJECTION DES CLUSTERS ---
df['Cluster'] = df['Ticker'].map(cluster_map)

# --- 4. CRÉATION DE LA TARGET (Horizon 5 jours) ---
df = df.sort_values(['Ticker', 'Date'])
df['Target_Price'] = df.groupby('Ticker')['Close'].shift(-5)
# 1 si ça monte dans 5 jours, 0 sinon
df['Target'] = (df['Target_Price'] > df['Close']).astype(int)

# --- 5. NETTOYAGE ET SAUVEGARDE ---
df_final = df.dropna(subset=['Target_Price'])
df_final.to_csv(OUTPUT_PATH, index=False)

print(f"✅ Superset créé avec succès !")
print(f"📊 Taille du dataset : {df_final.shape}")

✅ Superset créé avec succès !
📊 Taille du dataset : (105829, 12)
